In [ ]:
import tensorflow as tf

In [ ]:
import nltk
import numpy as np

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("imdb", split="train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
texts = np.array(dataset['text'])

In [ ]:
labels = np.array(dataset['label'])

In [ ]:
texts = [review.lower() for review in texts]

In [ ]:
import re
def remove_htmltags(text):
  return re.sub(r'<.*?>', '', text)

In [ ]:
texts = [remove_htmltags(review) for review in texts]

In [ ]:
def remove_special_chars(text):
  return re.sub(r'[-+=*@_#`"$%^&*[\]\()<>/\|}{~:]', '', text)

In [ ]:
texts = [remove_special_chars(review) for review in texts]

In [ ]:
texts = [' '.join(review.split()).strip().lower() for review in texts]

In [ ]:
#Decontraction of text
def removeShortForms(text):
    """
    Returns decontracted phrases
    """
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [ ]:
texts = [removeShortForms(review) for review in texts]

In [ ]:
texts[0]

'i rented i am curiousyellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it was seized by u.s. customs if it ever tried to enter this country, therefore being a fan of films considered controversial i really had to see this for myself.the plot is centered around a young swedish drama student named lena who wants to learn everything she can about life. in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.what kills me about i am curiousyellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it is not shot like 

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
def lemmatize(text):
  lem = nlp(text)
  return [''.join(token.lemma_) for token in lem]

In [ ]:
texts = [lemmatize(text) for text in texts]

In [ ]:
lemmatize('i am a very good runner and better dancer')

['I', 'be', 'a', 'very', 'good', 'runner', 'and', 'well', 'dancer']

In [ ]:
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

In [ ]:
texts = [word_tokenize(review) for review in texts]

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
excl = stopwords.words('english')

In [ ]:
parsedTexts = [[word for word in review if word not in excl] for review in texts]

In [ ]:
texts = [' '.join(review) for review in parsedTexts]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [ ]:
tokens = [tokenizer.texts_to_sequences([review])[0] for review in texts]

In [ ]:
max_len = max([len(review) for review in texts])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
padded_vectors = pad_sequences(tokens, maxlen=max_len, padding='pre')

In [ ]:
noofwords = len(tokenizer.word_index)+1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout

In [ ]:
model = Sequential()
model.add(Embedding(noofwords, 100, input_length=max_len))
model.add(GRU(100))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(texts, labels, epochs=2)

Epoch 1/2
356/782 ━━━━━━━━━━━━━━━━━━━━ 2:23:16 20s/step - accuracy: 0.7040 - loss: 0.5337